<a href="https://colab.research.google.com/github/IamJk28/aircraftgit/blob/main/Training_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Notebook


In this notebook we will train a model from TF2 Object Detection zoo/API on our aircraft images/Annotations

Give the model you are making a name:

In [ ]:
CUSTOM_MODEL_NAME = 'ssd_mobilenet'


In [ ]:
import os

# this cell allows the google colab session to access your drive contents where we have uploaded images and Annotations
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Go to https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

The models listed there can be plugged into our notebook, by copying a given model's link and pasting it into the PRETRAINED_MODEL_URL variable below

In [ ]:
# Adjust variable names appropriately - using the TF2 Object detection zoo @ https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

#paste the model link from tf2 zoo
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz'#Change for different model, get from tf2 zoo

# Extract the filename from the URL
filename = os.path.basename(PRETRAINED_MODEL_URL)

# Remove the file extension (.tar.gz) from the filename
model_name = os.path.splitext(filename)[0] #if this extraction of filename doesnt work just paste it in (e.g for PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz' , PRETRAINED_MODEL_NAME = ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8)

PRETRAINED_MODEL_NAME = model_name.split('.tar')[0]

PRETRAINED_MODEL_NAME

'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8'

In [ ]:
#choose the name of the image dataset you want to use  currently using 'final_dataset.tar.gz', a tar.gz file containing images/annotations
image_dataset = 'final_dataset.tar.gz'

In [ ]:
#don't edit
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

image resizing:
images should be resized so that they are the right input size for the model used. for example, for the model 'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8' , the input model size is 640x640 , so below, you would set new_width = 640 etc etc.

in the following cells the images and annotations will be automatically resized accordingly.


In [ ]:
new_width = 640
new_height = 640

Training Hyperparameters:

here we can edit the value of any variables/hyperparameters for our training run:

-training_steps: how long the model trains for

-batch_size: how many images the model stores in the GPU Ram at any given time whilst training. more = smoother training, but requires more gpu ram - too many = crash/error!


you can edit the value of any other hyperparameters by opening the config file in section 7 of this notebook

In [ ]:
training_steps = 20000
batch_size = 40

# 0. Setup Paths

In [ ]:
folderpaths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'TRAIN_PATH':os.path.join('Tensorflow','workspace','images','train'),
    'TEST_PATH':os.path.join('Tensorflow','workspace','images','test'),
    'COLLECT_PATH':os.path.join('Tensorflow','workspace','images','collectedimages')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(folderpaths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(folderpaths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
for path in folderpaths.values():
    if not os.path.exists(path):
      !mkdir -p {path}


#1. Untar Image and label Archive and move image and label folders to correct location

In [ ]:
#replace source_folder0 with link to tar.gz file of images and label folders

import shutil

source_folder0 = "drive/MyDrive/aircraftimage/final_dataset.tar.gz"
destination_folder0 = "Tensorflow/workspace/images"

if os.path.exists(source_folder0):
   shutil.copy(source_folder0, destination_folder0)

In [ ]:
#replace source_folder0 with the same tar.gz link/path as above

import tarfile

# Path to the archive file
source_folder0 = f"/content/Tensorflow/workspace/images/{image_dataset}"

# Extract the contents of the archive
with tarfile.open(source_folder0, 'r:gz') as tar:
    tar.extractall("/content/Tensorflow/workspace/images")


In [ ]:
#use the first two source_folder strings in future. upload the images and Annotations files from Microsoft teams INTO a new file (here we have called it "aircraftimage" ,change the path of source folders if you call it something else)

#this code takes the "images" and "Annotations" files that you have uploaded into Google Drive and copies them in to the directory Tensorflow/workspace/collectedimages .from here using some later process/code we split the collected images(and xml files) into training and testing sets

#again make sure that the source_folder 1 and 2 paths are relevant to the tar.gz file that was just untarred.

source_folder1 = "Tensorflow/workspace/images/images"
source_folder2 = "Tensorflow/workspace/images/Annotations"
destination_folder = "Tensorflow/workspace/images/collectedimages"

source_folders = [source_folder1, source_folder2]

for source_folder in source_folders:
    source_files = os.listdir(source_folder)
    for file_name in source_files:
        source_file = os.path.join(source_folder, file_name)
        destination_file = os.path.join(destination_folder, file_name)
        shutil.copy(source_file, destination_file)

In [ ]:
#check the number of images you are running this experiment with

#define the directory you want to count images in
directory_path1 = "/content/Tensorflow/workspace/images/images"
directory_path2 = "/content/Tensorflow/workspace/images/Annotations"

#list all images in the directory
items = os.listdir(source_folder1)

#count
num_items = len(items)

#print the number of images you are working with
print(f"Number of items in the directory: {num_items}")

#list all images in the directory
items = os.listdir(source_folder2)

#count
num_items = len(items)

#print the number of images you are working with
print(f"Number of items in the directory: {num_items}")


Number of items in the directory: 2279
Number of items in the directory: 2314


# Optional resizing step -- We can resize images and labels to match the correct input size for the model (if the chosen model does not have an inbuilt image_resizer, and if this hasn't already been done locally)

In [ ]:
import cv2
import os

def resize_images(input_folder, output_folder, new_width, new_height):
    """Iterates through all images in a given input folder (input_folder) and resizes them to a specified size.
    run annotation_converter.py afterwards to convert any existing/outstanding xml files to the proper size.
    """
    if not os.path.exists(output_folder):
      os.makedirs(output_folder)


    for filename in os.listdir(input_folder):
      if filename.endswith('JPG'):
        input_path = os.path.join(input_folder, filename)
        image = cv2.imread(input_path)
        if image is not None:
          # Resize the image
          resized_image = cv2.resize(image, (new_width, new_height))

          # Save the resized image to the output folder
          output_path = os.path.join(output_folder, filename)
          cv2.imwrite(output_path, resized_image)

In [ ]:
#uncheck this if you want to resize images in the cloud
resize_images('Tensorflow/workspace/images/collectedimages', 'Tensorflow/workspace/images/resizedcollectedimages', new_width, new_height)

In [ ]:
import cv2
import os
import xml.etree.ElementTree as ET

def resize_xml_annotations(xml_folder, output_folder, new_size):
    """ Iterates through all xml_files in a given input folder (xml_folder variable)
    and resizes them to map to the same location on a resized image.
    """
    new_width, new_height = new_size

    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_folder, xml_file)
            tree = ET.parse(xml_path)
            root = tree.getroot()

            image_size = root.find('size')
            image_width = int(image_size.find('width').text)
            image_height = int(image_size.find('height').text)

            width_ratio = new_width / image_width
            height_ratio = new_height / image_height

            for object in root.findall('object'):
                bndbox = object.find('bndbox')

                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                xmin = int(xmin * width_ratio)
                ymin = int(ymin * height_ratio)
                xmax = int(xmax * width_ratio)
                ymax = int(ymax * height_ratio)

                bndbox.find('xmin').text = str(xmin)
                bndbox.find('ymin').text = str(ymin)
                bndbox.find('xmax').text = str(xmax)
                bndbox.find('ymax').text = str(ymax)

            # Update the image size in the XML file
            image_size.find('width').text = str(new_width)
            image_size.find('height').text = str(new_height)

            # Save the updated XML file to the output folder
            output_path = os.path.join(output_folder, xml_file)
            tree.write(output_path)

In [ ]:
#uncheck if you want to resize xml labels in the cloud
resize_xml_annotations('Tensorflow/workspace/images/collectedimages', 'Tensorflow/workspace/images/resizedcollectedimages', (new_width,new_height))

#2. Train/Test Split

In [ ]:
import os
import random
from sklearn.model_selection import train_test_split
from shutil import copyfile

In [ ]:

# Set the paths to your image and XML directories #resized
image_directory ='Tensorflow/workspace/images/resizedcollectedimages'
xml_directory ='Tensorflow/workspace/images/resizedcollectedimages'

# Create the train and test directories if they don't exist
train_directory = 'Tensorflow/workspace/images/train'
test_directory = 'Tensorflow/workspace/images/test'
os.makedirs(train_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)

# List all image files in the image directory
image_files = [file for file in os.listdir(image_directory) if file.endswith('.JPG')]

# Set the random seed for reproducibility
random_seed = 42

# Perform the train/test split
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=random_seed)


# Move the selected images and their corresponding XML files to the train folder
for file in train_files:
    image_src = os.path.join(image_directory, file)
    #print(image_src)
    xml_src = os.path.join(xml_directory, file.replace('.JPG', '.xml'))
    #print(xml_src)
    image_dest = os.path.join(train_directory, file)
    #print(image_dest)
    xml_dest = os.path.join(train_directory, file.replace('.JPG', '.xml'))
    #print(xml_dest)
    if os.path.exists(xml_src):
      copyfile(image_src, image_dest)
      copyfile(xml_src, xml_dest)

# Move the remaining images and their corresponding XML files to the test folder
for file in test_files:
    image_src = os.path.join(image_directory, file)
    xml_src = os.path.join(xml_directory, file.replace('.JPG', '.xml'))
    image_dest = os.path.join(test_directory, file)
    xml_dest = os.path.join(test_directory, file.replace('.JPG', '.xml'))
    if os.path.exists(xml_src):
      copyfile(image_src, image_dest)
      copyfile(xml_src, xml_dest)


#examine train/test split by unchecking below
print("size of training set",len(train_files))
print("size of testing set",len(test_files))

#Number of images being used in this dataset
print("Overall Number of Images Input into Model: " + str(len(train_files) + len(test_files)))

#3. Download Tensorflow Object Detection API and Models

In [ ]:
#This downloads the object detection API
if not os.path.exists(os.path.join(folderpaths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {folderpaths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 89900, done.
remote: Counting objects: 100% (3612/3612), done.
remote: Compressing objects: 100% (1929/1929), done.
remote: Total 89900 (delta 1914), reused 3279 (delta 1645), pack-reused 86288
Receiving objects: 100% (89900/89900), 606.56 MiB | 16.22 MiB/s, done.
Resolving deltas: 100% (63812/63812), done.


In [ ]:
#Converts .protoc files into readable .py format
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.
Processing /content/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#This seems to be the necessary version of tensorflow (doesn't seem to work with tensorflow==2.14.0)
!pip install tensorflow==2.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 47.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.4
    Uninstalling gast-0.5.4:
      Successfully uninstalled gast-0.5.4
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.

If the output of the following cell has OK at the very bottom (in some fashion) then you have successfully installed TF2 Object Detection API

In [ ]:
VERIFICATION_SCRIPT = os.path.join(folderpaths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# verify if the installation has worked
!python {VERIFICATION_SCRIPT}

2023-11-18 17:37:20.951814: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-18 17:37:21.005320: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-18 17:37:22.025000: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2023-11-18 17:37:25.849406: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the T

In [ ]:
import object_detection

In [ ]:
#This downloads the actual model you plan to use from TF2 model zoo and moves it
!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {folderpaths['PRETRAINED_MODEL_PATH']}
!cd {folderpaths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2023-11-18 17:38:00--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.130.207, 74.125.68.207, 64.233.170.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.130.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90453990 (86M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v1_fp 100%[===================>]  86.26M  14.6MB/s    in 7.0s    

2023-11-18 17:38:08 (12.4 MB/s) - ‘ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [90453990/90453990]

ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.

# 4. Create Label Mapping

In [ ]:
#The machine will class objects as numbers, so we introduce label mapping to map those numbers back to recognisable names of our distinct object groups
labels = [{'name':'Aircraft', 'id':1},{'name':'Non-Aircraft', 'id':2}]
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

#5. Create TF records files

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {folderpaths['SCRIPTS_PATH']}

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [ ]:
#This code converts our xml annotations into a machine readable TFRecord file format
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(folderpaths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(folderpaths['ANNOTATION_PATH'], 'train.record')}
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(folderpaths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(folderpaths['ANNOTATION_PATH'], 'test.record')}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 6. Copy Model Configuration File to Training Folder

In [ ]:
!cp {os.path.join(folderpaths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(folderpaths['CHECKPOINT_PATH'])}

# 7. Update Config For Transfer Learning

This section is where we adjust hyperparameters of our model either to configure the model to simply work on our input data, or to optimise it for best performance.

It is important to read the config file carefully

In [ ]:

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.train_config.batch_size = batch_size
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(folderpaths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(folderpaths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(folderpaths['ANNOTATION_PATH'], 'test.record')]

Important: Change the below {model_name} pipeline_config.model.{model_name}.num_classes to the name listed in the general config file (e.g for an ssd, we write

pipeline_config.model.ssd.num_classes = len(labels)

- to explain, this sets the number of classes that the model will study to the size of the labels variable that we wrote in section 4


In [ ]:
#change {model_name} pipeline_config.model.{model_name}.num_classes to the name listed in the general config file
pipeline_config.model.ssd.num_classes = len(labels)


In [ ]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v1_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   

#8. Train the model

We specify the number of training steps we want to have the model learn for

In [ ]:
TRAINING_SCRIPT = os.path.join(folderpaths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TRAINING_SCRIPT, folderpaths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'],training_steps)

In [ ]:
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/ssd_mobilenet --pipeline_config_path=Tensorflow/workspace/models/ssd_mobilenet/pipeline.config --num_train_steps=20000


In [ ]:
!{command}

2023-11-18 17:38:24.081316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-11-18 17:38:27.427972: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I1118 17:38:27.429616 136149638571136 mirrored_strategy.py:419] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 20000
I1118 17:38:27.452880 136149638571136 config_util.py:552] Maybe overwriting train_steps: 20000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1118 17:38:27.453038 136149638571136 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W1118 17:38:27.479532 136149638571136 d

In [ ]:

# Specify the file path
file_path = f'/content//Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}/hyperparameters_and_variables.txt'

# Open the file in write mode ('w')
with open(file_path, 'w') as file:
    file.write("Hyperparameters and variables used to train/test this model\n")
    file.write(" Training variables and Hyperparameters:\n")
    file.write(f"  Batch size: {batch_size}\n")
    file.write(f"  Model Architecture: {PRETRAINED_MODEL_NAME}\n")
    file.write(f"  Training Steps: {training_steps}\n")

print(f'File "{file_path}" has been created and written to.')

In [ ]:
import shutil
import os

# Replace these paths with your source and destination folder paths
source_folder = f"/content/Tensorflow/workspace/models/{CUSTOM_MODEL_NAME}"
destination_folder = f"/content/drive/MyDrive/aircraftimage/trained_models/{CUSTOM_MODEL_NAME}"

# Check if the destination folder already exists
counter = 1
while os.path.exists(destination_folder):
    # If it exists, increment the counter and update the destination folder name
    destination_folder = f"{base_destination_folder}{CUSTOM_MODEL_NAME}_{counter}"
    counter += 1

# Create the destination folder
#os.makedirs(destination_folder)

# Copy the entire folder
shutil.copytree(source_folder, destination_folder)

print(f"Trained Model, testing records, checkpoints etc were copied to {destination_folder}")

'/content/drive/MyDrive/aircraftimage/temp/ssd_mobilenet_1'

Now open up the Inferencing Notebook and load the model you've just built - located at destination_folder

In [ ]:
destination_folder